Step1. Install Pytesseract and tesseract-OCR in Google Colab.

In [1]:
!sudo apt install tesseract-ocr

!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install requests fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149857 sha256=b30d5fb4a409232f58aa26e312e3a43d1d084eff6c38468aa922c04c10494a61
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


Step2. import libraries

In [13]:
import pytesseract
import shutil
import os
import random
#try:
from PIL import Image
#except ImportError:
#import Image

In [16]:
import numpy as np
#import pytesseract
from fuzzywuzzy import fuzz
from PIL import Image, ImageDraw, ImageOps
from PIL import ImageFont

In [28]:
def normalize(img):
    """
    Linear histogram normalization
    """
    arr = np.array(img, dtype=float)

    arr = (arr - arr.min()) * (255 / arr[:, :50].min())
    arr[arr > 255] = 255
    arr[arr < 0] = 0

    return Image.fromarray(arr.astype('uint8'), 'L')

In [29]:
def evaluateImage(imageFile, trueTextFile):
    with open(trueTextFile, 'r') as f:
        trueText = f.readlines()

    # remome \n character
    trueText = [text.rstrip() for text in trueText]

    # load image and convert to grayscale
    img = Image.open(imageFile)
    # img.show()
    img = ImageOps.grayscale(img)
    # img.show()
    w, h = img.size

    img = normalize(img)
    # img.show()

    # resize image to improve OCR
    img = img.resize((int(w / 2), int(h / 2)))
    w, h = img.size

    # run OCR
    options = r'--oem 1 --psm 6 -c load_system_dawg=false -c load_freq_dawg=false  -c textord_old_xheight=0  -c textord_min_xheight=100 -c ' \
              r'preserve_interword_spaces=0'
    OCRtext = pytesseract.image_to_string(img, config=options)

    # debug mode
    showOCRImage = False
    if showOCRImage:
        boxes = pytesseract.image_to_boxes(img, config=options)
        draw = ImageDraw.Draw(img)

        font = ImageFont.truetype('./verdanaRef.ttf', 30)
        for b in boxes.splitlines():
            text = b[0]
            b = [int(x) for x in b.split(' ')[1:]]
            draw.rectangle([b[0], h - b[1], b[2], h - b[3]], fill=None, outline='#ff0000', width=1)
            draw.text((b[0], h - b[1]), text, font=font, fill=0)

        img.show()

    # removes form feed character  \f
    OCRtext = OCRtext.replace('\n\f', '').replace('\n\n', '\n')

    # split lines
    OCRtext = OCRtext.split('\n')

    # remove empty lines
    OCRtext = [x.strip() for x in OCRtext if x.strip()]

    # check if OCR extracted 3 lines of text
    print('File:' + imageFile)
    print('True text (middle line): %s' % trueText[1])

    if len(OCRtext) != 3:
        print('ERROR: OCR text does not have 3 lines of text!')
        print(OCRtext)
        return None
    else:
        score = fuzz.ratio(trueText[1], OCRtext[1])
        print('OCR  text (middle line): %s' % OCRtext[1])
        print('Score: %d' % score)

        return float(score)


if __name__ == "__main__":

    InputFile = '/content/drive/MyDrive/example.png'
    trueText = '/content/drive/MyDrive/example_TrueText.txt'

In [30]:
#InputFile = '/content/drive/MyDrive/predicted9.png'
InputFile = '/content/drive/MyDrive/focusStep_5_verdanaRef_size_30_sample_0039.png'
trueText = '/content/drive/MyDrive/text_step5/focusStep_5_verdanaRef_size_30_sample_0039.txt'

#InputFile = '/content/drive/MyDrive/example.png'
#trueText = '/content/drive/MyDrive/example_TrueText.txt'

score = evaluateImage(InputFile, trueText)

File:/content/drive/MyDrive/focusStep_5_verdanaRef_size_30_sample_0039.png
True text (middle line): uuaAGbZtkf
ERROR: OCR text does not have 3 lines of text!
[]


In [31]:
InputFile = '/content/drive/MyDrive/example.png'
trueText = '/content/drive/MyDrive/example_TrueText.txt'

score = evaluateImage(InputFile, trueText)

File:/content/drive/MyDrive/example.png
True text (middle line): RcsijkFLTX
OCR  text (middle line): RcsijkFLTX
Score: 100
